<a href="https://colab.research.google.com/github/wlin988/Colab/blob/master/Load_em_to_My_GDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
%cd /content/drive/My Drive
!ls -lta

/content/drive/My Drive
total 84850
drwx------ 2 root root     4096 Oct 18 07:19 'Colab Notebooks'
-rw------- 1 root root      151 Jul 12 03:49 'WayneLin Resume.gdoc'
-rw------- 1 root root    22157 Jul 12 03:49 'WayneLin Resume.docx'
-rw------- 1 root root 81676760 Aug 16  2018  GitHubDesktopSetup.exe
-rw------- 1 root root  1238751 Aug 16  2018  LeetCode-master.zip
-rw------- 1 root root   453979 Aug 16  2018 'GitHub - kamyu104_LeetCode_ Python _ C++ 11 Solutions of All LeetCode Questions.html'
-rw------- 1 root root  1130840 Aug 16  2018  installbackupandsync.exe
-rw------- 1 root root      179 Nov  2  2017  ATT00001.htm
-rw------- 1 root root  1786772 Apr  3  2017 'HN - Benefits Overview (1).pdf'
-rw------- 1 root root    13046 Nov 16  2016 'JD - Req#124618BR - Software Engineer, Cloud.docx'
-rw------- 1 root root      151 Aug 26  2016 'Untitled document.gdoc'
-rw------- 1 root root      119 Jun  5  2015 'My Saved Places.gmap'
-rw------- 1 root root    21267 Jan 13  2014 'Invention

In [8]:
!git clone https://github.com/joosthub/PyTorchNLPBook.git

Cloning into 'PyTorchNLPBook'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 7.91 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [6]:
import os
from getpass import getpass
import urllib

cmd = 'git clone https://github.com/joosthub/PyTorchNLPBook.git'
os.system(cmd)

32768

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

ModuleNotFoundError: ignored

In [0]:
class PreTrainedEmbeddings(object):
    """ A wrapper around pre-trained word vectors and their use """
    def __init__(self, word_to_index, word_vectors):
        """
        Args:
            word_to_index (dict): mapping from word to integers
            word_vectors (list of numpy arrays)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("Building Index!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("Finished!")
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """Instantiate from pre-trained vector file.
        
        Vector file should be of the format:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        Args:
            embedding_file (str): location of the file
        Returns: 
            instance of PretrainedEmbeddigns
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
                
        return cls(word_to_index, word_vectors)
    
    def get_embedding(self, word):
        """
        Args:
            word (str)
        Returns
            an embedding (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]

    def get_closest_to_vector(self, vector, n=1):
        """Given a vector, return its n nearest neighbors
        
        Args:
            vector (np.ndarray): should match the size of the vectors 
                in the Annoy index
            n (int): the number of neighbors to return
        Returns:
            [str, str, ...]: words that are nearest to the given vector. 
                The words are not ordered by distance 
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_and_print_analogy(self, word1, word2, word3):
        """Prints the solutions to analogies using word embeddings

        Analogies are word1 is to word2 as word3 is to __
        This method will print: word1 : word2 :: word3 : word4
        
        Args:
            word1 (str)
            word2 (str)
            word3 (str)
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)

        # now compute the fourth word's embedding!
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship

        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = set([word1, word2, word3])
        closest_words = [word for word in closest_words 
                             if word not in existing_words] 

        if len(closest_words) == 0:
            print("Could not find nearest neighbors for the computed vector!")
            return
        
        for word4 in closest_words:
            print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

In [0]:
embeddings = PreTrainedEmbeddings.from_embeddings_file('data/glove/glove.6B.100d.txt')

In [0]:
embeddings.compute_and_print_analogy('man', 'he', 'woman')

In [0]:
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

In [0]:
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

In [0]:
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

In [0]:
embeddings.compute_and_print_analogy('leg', 'legs', 'hand')

In [0]:
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

In [0]:
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

In [0]:
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

In [0]:
embeddings.compute_and_print_analogy('man', 'king', 'woman')

In [0]:
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

In [0]:
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')